In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [71]:
zestien = pd.read_csv('data_raw_2021.csv', index_col=0).rename(columns={'index': 'rider'})

In [72]:
zestien

,rider,01-stage,01-gc,01-points,01-youth,02-stage,02-gc,02-points,02-youth,03-stage,...,13-teams,14-teams,15-teams,16-teams,17-teams,18-teams,19-teams,20-teams,21-teams,year
0,Ganna Filippo,1,1,1.0,1.0,20,1.0,6.0,1.0,51.0,...,1,1,2,2.0,1.0,2.0,1.0,1.0,1.0,2021
1,Affini Edoardo,2,2,2.0,2.0,101,2.0,8.0,2.0,131.0,...,7,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021
2,Foss Tobias,3,3,3.0,3.0,100,3.0,11.0,3.0,50.0,...,7,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021
3,Almeida João,4,4,4.0,4.0,50,5.0,14.0,5.0,34.0,...,6,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021
4,Cavagna Rémi,5,5,5.0,NaN,79,6.0,16.0,NaN,62.0,...,6,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,Fortunato Lorenzo,180,180,NaN,49.0,153,175.0,NaN,49.0,45.0,...,16,15,15,14.0,14.0,13.0,13.0,13.0,13.0,2021
180,Viviani Attilio,181,181,NaN,50.0,91,176.0,NaN,50.0,169.0,...,20,21,20,21.0,21.0,21.0,21.0,22.0,22.0,2021
181,Tagliani Filippo,182,182,NaN,NaN,145,177.0,7.0,NaN,154.0,...,21,19,19,19.0,19.0,18.0,18.0,19.0,19.0,2021
182,Niv Guy,183,183,NaN,NaN,106,178.0,NaN,NaN,93.0,...,17,16,16,17.0,17.0,16.0,17.0,17.0,17.0,2021


In [73]:
stage_s = list(np.arange(2,32,2))+list(np.arange(32,48,4))+[50]
stage_s_i = list(np.arange(1,21,1))
stage_s_dict = dict(zip( stage_s_i, stage_s[::-1]))
stage_s_dict

{1: 50,
 2: 44,
 3: 40,
 4: 36,
 5: 32,
 6: 30,
 7: 28,
 8: 26,
 9: 24,
 10: 22,
 11: 20,
 12: 18,
 13: 16,
 14: 14,
 15: 12,
 16: 10,
 17: 8,
 18: 6,
 19: 4,
 20: 2}

In [74]:
stage_gc = list(np.arange(2,12,2))
stage_gc_i = list(np.arange(1,6,1))
stage_gc_dict = dict(zip( stage_gc_i, stage_gc[::-1]))
stage_gc_dict

{1: 10, 2: 8, 3: 6, 4: 4, 5: 2}

In [75]:
stage_points = [1]+list(np.arange(2,10,2))
stage_points_i = list(np.arange(1,6,1))
points_dict = dict(zip( stage_points_i,  stage_points[::-1]))
points_dict

{1: 8, 2: 6, 3: 4, 4: 2, 5: 1}

In [76]:
stage_kom = [1]+list(np.arange(2,10,2))
stage_kom_i = list(np.arange(1,6,1))
kom_dict = dict(zip( stage_kom_i,  stage_kom[::-1]))
kom_dict

{1: 8, 2: 6, 3: 4, 4: 2, 5: 1}

In [77]:
stage_youth = list(np.arange(1,5,1))+[6]
stage_youth_i = list(np.arange(1,6,1))
youth_dict = dict(zip( stage_youth_i,  stage_youth[::-1]))
youth_dict

{1: 6, 2: 4, 3: 3, 4: 2, 5: 1}

In [78]:
zestien['sum'] = 0
for i in zestien.columns:
    print(i)
    if '-stage' in i:
        zestien[i] = zestien[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        zestien[i] = zestien[i].map(stage_s_dict).fillna(0)
        zestien['sum'] = zestien[i] + zestien['sum']
    elif '-gc' in i:
        zestien[i] = zestien[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        zestien[i] = zestien[i].map(stage_gc_dict).fillna(0)
        zestien['sum'] = zestien['sum']+ zestien[i]
    elif '-points' in i:
        zestien[i] = zestien[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        zestien[i] = zestien[i].map(points_dict).fillna(0)
        zestien['sum'] = zestien['sum']+ zestien[i]
    elif '-youth' in i:
        zestien[i] = zestien[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        zestien[i] = zestien[i].map(youth_dict).fillna(0)
        zestien['sum'] = zestien['sum']+ zestien[i]
    elif '-kom' in i:
        zestien[i] = zestien[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        zestien[i] = zestien[i].map(kom_dict).fillna(0)
        zestien['sum'] = zestien['sum']+ zestien[i]
    else:
        pass

rider
01-stage
01-gc
01-points
01-youth
02-stage
02-gc
02-points
02-youth
03-stage
03-gc
03-points
03-youth
04-stage
04-gc
04-points
04-youth
05-stage
05-gc
05-points
05-youth
06-stage
06-gc
06-points
06-youth
07-stage
07-gc
07-points
07-youth
08-stage
08-gc
08-points
08-youth
09-stage
09-gc
09-points
09-youth
10-stage
10-gc
10-points
10-youth
11-stage
11-gc
11-points
11-youth
12-stage
12-gc
12-points
12-youth
13-stage
13-gc
13-points
13-youth
14-stage
14-gc
14-points
14-youth
15-stage
15-gc
15-points
15-youth
16-stage
16-gc
16-points
16-youth
17-stage
17-gc
17-points
17-youth
18-stage
18-gc
18-points
18-youth
19-stage
19-gc
19-points
19-youth
20-stage
20-gc
20-points
20-youth
21-stage
21-gc
21-points
21-youth
14-kom
15-kom
16-kom
17-kom
18-kom
19-kom
20-kom
21-kom
06-kom
07-kom
08-kom
09-kom
10-kom
11-kom
12-kom
13-kom
04-kom
05-kom
03-kom
02-kom
team
01-teams
02-teams
03-teams
04-teams
05-teams
06-teams
07-teams
08-teams
09-teams
10-teams
11-teams
12-teams
13-teams
14-teams
15-teams


In [79]:
zestien[sorted(zestien.columns)].sort_values('sum')

,01-gc,01-points,01-stage,01-teams,01-youth,02-gc,02-kom,02-points,02-stage,02-teams,...,21-gc,21-kom,21-points,21-stage,21-teams,21-youth,rider,sum,team,year
91,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,3,...,0.0,0.0,0.0,0.0,8.0,0.0,Serry Pieter,0.0,Deceuninck - Quick Step,2021
109,0.0,0.0,0.0,13,0.0,0.0,0.0,0.0,0.0,13,...,0.0,0.0,0.0,0.0,12.0,0.0,Duchesne Antoine,0.0,Groupama - FDJ,2021
108,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,11,...,0.0,0.0,0.0,0.0,16.0,0.0,Benedetti Cesare,0.0,BORA - hansgrohe,2021
105,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,11,...,0.0,0.0,0.0,0.0,16.0,0.0,Oss Daniel,0.0,BORA - hansgrohe,2021
104,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,4,...,0.0,0.0,0.0,0.0,17.0,0.0,Neilands Krists,0.0,Israel Start-Up Nation,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,4.0,2.0,36.0,3,2.0,2.0,0.0,0.0,0.0,3,...,0.0,0.0,0.0,32.0,8.0,2.0,Almeida João,306.0,Deceuninck - Quick Step,2021
50,0.0,0.0,0.0,11,0.0,0.0,0.0,1.0,32.0,11,...,0.0,0.0,8.0,0.0,16.0,0.0,Sagan Peter,317.0,BORA - hansgrohe,2021
24,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,0.0,8,...,8.0,4.0,0.0,8.0,9.0,0.0,Caruso Damiano,361.0,Bahrain - Victorious,2021
10,0.0,0.0,20.0,6,0.0,0.0,0.0,0.0,0.0,6,...,4.0,0.0,0.0,4.0,4.0,4.0,Vlasov Aleksandr,405.0,Astana - Premier Tech,2021


In [192]:
team = []

for i in zestien.columns:
    if 'teams' in i:
        team.append(i)
        
        
#zestien.drop(columns=['Unnamed: 0'], inplace=True)
#zestien.drop(columns=team, inplace=True)
zestien

,Unnamed: 0,rider,1-stage,1-gc,1-points,1-youth,2-stage,2-gc,2-points,2-kom,...,14-teams,15-teams,16-teams,17-teams,18-teams,19-teams,20-teams,21-teams,year,sum
0,0,Ganna Filippo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,2,2.0,1.0,2.0,1.0,1.0,1.0,2021,0
1,1,Affini Edoardo,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021,0
2,2,Foss Tobias,0.0,0.0,2.0,3.0,0.0,0.0,0.0,2.0,...,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021,0
3,3,Almeida João,0.0,4.0,6.0,4.0,0.0,8.0,0.0,8.0,...,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021,0
4,4,Cavagna Rémi,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,...,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,179,Fortunato Lorenzo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15,15,14.0,14.0,13.0,13.0,13.0,13.0,2021,0
180,180,Viviani Attilio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21,20,21.0,21.0,21.0,21.0,22.0,22.0,2021,0
181,181,Tagliani Filippo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19,19,19.0,19.0,18.0,18.0,19.0,19.0,2021,0
182,182,Niv Guy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16,16,17.0,17.0,16.0,17.0,17.0,17.0,2021,0


In [11]:
zestien

,rider,1-stage,1-gc,1-points,2-stage,2-gc,2-points,3-stage,3-gc,3-points,...,13-teams,14-teams,15-teams,16-teams,17-teams,18-teams,19-teams,20-teams,21-teams,year
0,Dumoulin Tom,1,1,1.0,36,1,10.0,36,2.0,14.0,...,15,15,16,16,16,14,14,14,14,2016
1,Roglič Primož,2,2,2.0,61,2,12.0,174,165.0,19.0,...,11,10,10,9,9,10,10,11,11,2016
2,Amador Andrey,3,3,3.0,9,4,9.0,62,3.0,13.0,...,1,2,2,2,2,3,3,3,3,2016
3,Ludvigsson Tobias,4,4,4.0,22,5,17.0,17,4.0,20.0,...,15,15,16,16,16,14,14,14,14,2016
4,Kittel Marcel,5,5,5.0,1,3,1.0,1,1.0,1.0,...,5,5,5,6,6,6,7,6,6,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Grosu Eduard-michael,194,194,NaN,14,176,26.0,182,190.0,35.0,...,19,21,21,21,21,21,21,20,20,2016
194,Ruffoni Nicola,195,195,NaN,5,177,11.0,18,98.0,16.0,...,17,16,15,15,15,16,18,19,19,2016
195,Jim Songezo,196,196,NaN,183,196,NaN,193,193.0,NaN,...,9,9,9,10,10,9,9,9,9,2016
196,Pelucchi Matteo,197,197,NaN,23,178,NaN,13,100.0,30.0,...,20,19,19,19,19,20,19,18,18,2016
